## Readback uC pool 

In [1]:
# External imports 
import sys
# import serial
import pandas as pd

# Internal imports
sys.path.append('..')
import CRV 

In [2]:
# Initialise CRV class
crv = CRV.CRV("/dev/ttyUSB0", True)

In [3]:
# Print all attributes and methods of the class
print(dir(crv))

['LP', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'close', 'cmd', 'dcs', 'febAFEHighGain', 'febBias', 'febDdrStatus', 'febFPGA', 'febGetSample', 'febGetThr', 'febPageClear', 'febPageHeader', 'febPageRead', 'febPedestal', 'febPwrCycle', 'febReset', 'febSetAFEDAC', 'febSetBias', 'febSetGate', 'febSetThr', 'febSetup', 'getTriggers', 'lastUBs', 'markerBits', 'n_sample', 'page_size', 'parse', 'read', 'readDCS', 'readDReqTrace', 'readOutTrace', 'readOutput', 'readTriggers', 'readm', 'reset', 'resetLink', 'rocCnters', 'rocDdrRead', 'rocDdrReadEv', 'rocDdrReset', 'rocDdrStatus', 'rocDebBuffGet', 'rocDebBuffStatus', 'rocDebBuffTrig', 'rocDuty', 'rocFPGA', 'rocHeartbeat', 'rocLock', 'rocMa

In [62]:
# ROC uC pool addresses

par_names = [
    "Serial number (board ID)", "Most recent spill number", "Board temperature (degC)",
    "ADC channel 1: +1.2 V supply monitor", "ADC channel 2: +1.8 V supply monitor",
    "ADC channel 3: +5.0 V supply monitor", "ADC channel 4: +10 V supply monitor",
    "ADC channel 5: +2.5 V supply monitor", "ADC channel 6: -5.0 V supply monitor",
    "ADC channel 7: +15 V supply monitor", "ADC channel 8: +3.3 V supply monitor",
    "ADC channel 9: SiPM Bias Bus0", "ADC channel 10: SiPM Bias Bus1", 
    "ADC channel 11: SiPM Bias Bus2", "ADC channel 12: SiPM Bias Bus3",
    "ADC channel 13: SiPM Bias Bus4", "ADC channel 14: SiPM Bias Bus5", 
    "ADC channel 15: SiPM Bias Bus6", "ADC channel 16: SiPM Bias Bus7",
    "AFE DAC regs 23: Bias bus 1", "AFE DAC regs 24: Bias bus 2",
    "AFE (CMB) temperature 0", "AFE (CMB) temperature 2", 
    "AFE (CMB) temperature 3", "AFE (CMB) temperature 4",
    "AFE DAC regs 23: Bias bus 1", "AFE DAC regs 24: Bias bus 2",
    "AFE (CMB) temperature 0", "AFE (CMB) temperature 2", 
    "AFE (CMB) temperature 3", "AFE (CMB) temperature 4",
    "AFE DAC regs 23: Bias bus 1", "AFE DAC regs 24: Bias bus 2",
    "AFE (CMB) temperature 0", "AFE (CMB) temperature 2", 
    "AFE (CMB) temperature 3", "AFE (CMB) temperature 4",
    "AFE DAC regs 23: Bias bus 1", "AFE DAC regs 24: Bias bus 2",
    "AFE (CMB) temperature 0", "AFE (CMB) temperature 2", 
    "AFE (CMB) temperature 3", "AFE (CMB) temperature 4"
]

# Base address
base_reg = 0x8800

# Initialise list to store the rows
pool_regs = []

# Iterate thro' 43 addresses
for i in range(0, 43):

    # bits 0-6 are word address
    # bits 7-11 are needed for the port number
    # bits 12-15 are something else
    
    # Create a mask with bits 0 through 6 set to 1
    mask = 0b00000000001111111

    # Flip bits based on the current iteration
    word_reg = base_reg | i  # Base address OR'd with the current index

    block = ""
    if i < 19: 
        block = "FEB"
    elif i < 25: 
        block = "FPGA 0"
    elif i < 31:
        block = "FPGA 1"
    elif i < 37:
        block = "FPGA 2"
    else: 
        block = "FPGA 3"
    
    # Append the values
    pool_regs.append([block, par_names[i], hex(word_reg)])

df_pool = pd.DataFrame(pool_regs, columns=["block", "par", "reg"])
# display(df_pool)

# df_pool.to_csv("pool.csv", index=False)


In [31]:
# FEB info from ROC uC pool
print(crv.febBias().decode())

LC ADC
1.2v_Pos   1.21  ch00= 4BC
1.8v_Pos   1.80  ch01= 703
5.0v_Pos   5.01  ch02= 9C7
10v_Pos    9.87  ch03= 9A3
2.5v_Pos   2.51  ch04= 9D1
5.0v_Neg   4.96  ch05= 9B0
15v_Pos   14.89  ch06= 9B2
3.3v_Pos   3.33  ch07= D01
Bias_0    54.98  ch08= ABD
Bias_1    54.88  ch09= AB8
Bias_2    54.96  ch10= ABC
Bias_3    54.64  ch11= AAC
Bias_4    54.90  ch12= AB9
Bias_5    55.38  ch13= AD1
Bias_6    54.86  ch14= AB7
Bias_7    54.76  ch15= AB2
Temp_C    40.76




In [61]:
# Read a register 
# response = crv.read("8813", lc=True)
# response = crv.read("8814", lc=True)

def read_response(crv, reg, verbose=True):
    # Enable/disable printout
    crv.verbose = verbose
    # Read 
    response = crv.read(reg, lc=True)
    # Hex value
    hex_value = response[0]
    # Decimal value
    # numeric_value = int(response[0], 16) 
    # Print
    return response[0]
    print(f"{hex_value}, {numeric_value}")

# read_response(crv, "8813")

responses = []
for par, reg in zip(list(df_pool["par"]), list(df_pool["reg"])): 
    # print(f"\n{par}")
    hex_value = read_response(crv, reg[2:], False)
    numeric_value = int(hex_value, 16) 
    responses.append(["0x"+hex_value, numeric_value])

df_response = pd.DataFrame(responses, columns=["hex_response", "int_response"])
df_comb = pd.concat([df_pool, df_response], axis=1)
display(df_comb)

,block,par,reg,hex_response,int_response
0,FEB,Serial number (board ID),0x8800,0x0000,0
1,FEB,Most recent spill number,0x8801,0x0000,0
2,FEB,Board temperature (degC),0x8802,0x0000,0
3,FEB,ADC channel 1: +1.2 V supply monitor,0x8803,0x7880,30848
4,FEB,ADC channel 2: +1.8 V supply monitor,0x8804,0x0800,2048
5,FEB,ADC channel 3: +5.0 V supply monitor,0x8805,0x13A0,5024
6,FEB,ADC channel 4: +10 V supply monitor,0x8806,0xD720,55072
7,FEB,ADC channel 5: +2.5 V supply monitor,0x8807,0x1218,4632
8,FEB,ADC channel 6: -5.0 V supply monitor,0x8808,0x012A,298
9,FEB,ADC channel 7: +15 V supply monitor,0x8809,0x0600,1536
